In [25]:
# ATTENTION: Please do not alter any of the provided code in the exercise. Only add your own code where indicated
# ATTENTION: Please do not add or remove any cells in the exercise. The grader will check specific cells based on the cell position.
# ATTENTION: Please use the provided epoch values when training.

# In this exercise you will train a CNN on the FULL Cats-v-dogs dataset
# This will require you doing a lot of data preprocessing because
# the dataset isn't split into training and validation for you
# This code block has all the required inputs
import os
import requests
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [13]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

--2021-12-03 18:00:46--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 2a02:26f0:e600:58d::e59, 2a02:26f0:e600:588::e59, 23.211.8.197
Connecting to download.microsoft.com (download.microsoft.com)|2a02:26f0:e600:58d::e59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M  6.09MB/s    in 2m 2s   

2021-12-03 18:02:49 (6.42 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [26]:
# This code block unzips the full Cats-v-Dogs dataset to /tmp
# which will create a tmp/PetImages directory containing subdirectories
# called 'Cat' and 'Dog' (that's how the original researchers structured it)
local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


In [27]:
os.listdir('/tmp')

['40888E9F-FB5A-4FAC-B5B0-53CFEA76F179',
 'com.adobe.AdobeIPCBroker.ctrl-renesalmon',
 'fseventsd-uuid',
 'VulcanConfig.data',
 'AFCAC02E-515D-4B4E-A3E9-9C6068676491_OUT',
 '8AB607D0-14B0-453B-9F6C-139D0F17BA72',
 'AlTest1.out',
 'com.google.Keystone',
 'com.apple.launchd.cehxDFHeNM',
 'powerlog',
 'AlTest1.err',
 'cats_and_dogs_filtered',
 'MSR-LA - 3467.docx',
 'cats_and_dogs_filtered.zip',
 'AFCAC02E-515D-4B4E-A3E9-9C6068676491_IN',
 'C4BBC5E4-8776-402F-9159-891647BE2CAD',
 'adobegc.log',
 'PetImages',
 'cats_and_dogs_filtered2.zip',
 'cats-and-dogs.zip',
 'ExmanProcessMutex',
 '4544BFFE-F102-4AA9-9BE5-735612F36128',
 'cats_and_dogs.zip',
 '6001466D-D870-40E6-8172-E8E8198AA4BD',
 'readme[1].txt']

In [28]:
os.listdir('/tmp/PetImages')

['Cat', 'Dog']

In [29]:
print(len(os.listdir('/tmp/PetImages/Cat')))
print(len(os.listdir('/tmp/PetImages/Dog')))

12501
12501


In [33]:
# Use os.mkdir to create your directories
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'
try:
    #YOUR CODE GOES HERE
    os.mkdir('tmp/cats-v-dogs/')
    os.mkdir('tmp/cats-v-dogs/training/')
    os.mkdir('tmp/cats-v-dogs/testing/')
    os.mkdir('tmp/cats-v-dogs/training/cats/')
    os.mkdir('tmp/cats-v-dogs/training/dogs/')
    os.mkdir('tmp/cats-v-dogs/testing/cats/')
    os.mkdir('tmp/cats-v-dogs/testing/dogs/')
    
except OSError:
    pass

In [36]:
# Write a python function called split_data which takes
# a SOURCE directory containing the files
# a TRAINING directory that a portion of the files will be copied to
# a TESTING directory that a portion of the files will be copie to
# a SPLIT SIZE to determine the portion
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# os.path.getsize(PATH) gives you the size of the file
# copyfile(source, destination) copies a file from source to destination
# random.sample(list, len(list)) shuffles a list
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
# YOUR CODE STARTS HERE
    
    Images = []

    #Randomize Data
    List_of_Images = os.listdir(SOURCE)
    Randomized_images = random.sample(List_of_Images, len(List_of_Images))
    
    #Split Data
    splitter = int(len(Randomized_images) * SPLIT_SIZE)
    
    #Copy images to directories
    for image in range(len(Randomized_images)):
        
        #Sort files with size 0 out
        if os.path.getsize(SOURCE+Randomized_images[image]) >0:
            Images.append(Randomized_images[image])
        
            if image > splitter:
                copyfile(SOURCE+Randomized_images[image], TRAINING+Randomized_images[image])
            
            else:
                copyfile(SOURCE+Randomized_images[image], TESTING+Randomized_images[image])

        else:
            print(f'{image} is 0 and excluded')

    # YOUR CODE ENDS HERE


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/cats-v-dogs/testing/cats/10812.jpg'

In [12]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 1350
# 1350
# 150
# 150

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/cats-v-dogs/training/cats/'

In [9]:
# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
model = tf.keras.models.Sequential([
# YOUR CODE HERE
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
                           
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 5 (1948258876.py, line 15)

# NOTE:

In the cell below you **MUST** use a batch size of 10 (`batch_size=10`) for the `train_generator` and the `validation_generator`. Using a batch size greater than 10 will exceed memory limits on the Coursera platform.

In [11]:
TRAINING_DIR = '/tmp/cats-v-dogs/training'
train_datagen = ImageDataGenerator(
                        rescale=1/255,
                        rotation_range=20,
                        shear_range=20,
                        zoom_range=20,
                        fill_mode='nearest'
                        horizontal_flip=True)
    

# NOTE: YOU MUST USE A BATCH SIZE OF 10 (batch_size=10) FOR THE 
# TRAIN GENERATOR.
train_generator = train_datagen.flow_from_directory(
                        TRAINING_DIR,
                        batch_size=20,
                        target_size=(150,150),
                        class_mode='binary')

VALIDATION_DIR = '/tmp/cats-v-dogs/training'
validation_datagen = ImageGenerator(rescale=1/255)

# NOTE: YOU MUST USE A BACTH SIZE OF 10 (batch_size=10) FOR THE 
# VALIDATION GENERATOR.
validation_generator = validation_datagen.flow_from_directory(
                        VALIDATION_DIR,    
                        batch_size=10,
                        target_size=(150,150),
                        class_mode='binary'
                        )

# Expected Output:
# Found 2700 images belonging to 2 classes.
# Found 300 images belonging to 2 classes.

SyntaxError: invalid syntax (4093951999.py, line 8)

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=2,
                              verbose=1,
                              validation_data=validation_generator)


In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

# Submission Instructions

In [ ]:
# Now click the 'Submit Assignment' button above.

# When you're done or would like to take a break, please run the two cells below to save your work and close the Notebook. This will free up resources for your fellow learners. 

In [ ]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();

In [ ]:
%%javascript
IPython.notebook.session.delete();
window.onbeforeunload = null
setTimeout(function() { window.close(); }, 1000);